### The Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/stars.csv')

In [3]:
df.sample(5)

,description,language,repo,stargazers,user
11993,A PHP Blogging Platform. Simple and Powerful.,PHP,typecho/typecho,6099,covein
7107,Idiomatic statistical operators for Kotlin,Kotlin,thomasnield/kotlin-statistics,634,edujtm
16600,NaN,Python,TristenHarr/MyPyBuilder,157,tones7778
6794,Basic messaging service in python,Python,zappala/messaging-service-python,6,DahlitzFlorian
902,A standard library for microservices.,Go,go-kit/kit,15067,enzoftware


### The Cleanup

In [4]:
df = df[df['repo'] != 'maxhumber/gazpacho']
df = df[df.language.isin(['Python', 'Jupyter Notebook'])]
popular = pd.DataFrame(df['repo'].value_counts())
select_repos = popular[popular['repo'] >= 5].index.tolist()
df = df[df['repo'].isin(select_repos)]

In [5]:
df.sample(5)

,description,language,repo,stargazers,user
12458,Parallel computing with task scheduling,Python,dask/dask,5688,jamiehall007
12520,Python 3.7 program to search Library Genesis v...,Python,xSetrox/pytextbooks,32,greed2411
18399,"A curated list of awesome Python frameworks, l...",Python,vinta/awesome-python,74026,berlotto
13618,Glances an Eye on your system. A top/htop alte...,Python,nicolargo/glances,14294,simms21
2098,The pytest framework makes it easy to write sm...,Python,pytest-dev/pytest,4924,stribny


### The Preparation

In [6]:
df = df.groupby(['user'])['repo'].apply(lambda x: ','.join(x))
df = pd.DataFrame(df)

In [7]:
df.sample(5)

,repo
user,
BudSun,"30-seconds/30-seconds-of-python,jackfrued/Pyth..."
checktheroads,"kivy/kivy,minimaxir/big-list-of-naughty-string..."
iannil,"falconry/falcon,django-oscar/django-oscar,Mise..."
bnikanjam,"pypa/pipenv,pydanny/cookiecutter-django,django..."
fly51fly,"sebst/pythonic-news,pytorch/captum,wxWidgets/P..."


### The Model

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors


class NNRecommender:
    def __init__(self, n_neighbors=10, max_features=1000, tokenizer=lambda x: x.split(",")):
        self.cv = CountVectorizer(tokenizer=tokenizer, max_features=max_features)
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)

    def fit(self, X):
        self.X = X
        X = self.cv.fit_transform(X)
        self.nn.fit(X)
        return self

    def predict(self, X):
        Xp = []
        for Xi in X:
            Xt = self.cv.transform([Xi])
            _, neighbors = self.nn.kneighbors(Xt)
            repos = []
            for n in neighbors[0]:
                r = self.X.iloc[int(n)].split(",")
                repos.extend(r)
            repos = list(set(repos))
            repos = [r for r in repos if r not in Xi.split(",")]
            Xp.append(repos)
        return Xp

In [9]:
model = NNRecommender()
model.fit(df['repo'])

In [10]:
df['repo'][42]

'CorentinJ/Real-Time-Voice-Cloning,s0md3v/goop,psf/requests,slundberg/shap,facebookresearch/pytext,practicalAI/practicalAI,Zulko/moviepy,python/mypy,ParthS007/background,keon/algorithms,google/python-fire,eriklindernoren/ML-From-Scratch,minimaxir/big-list-of-naughty-strings,pudo/dataset,nvbn/thefuck,dbcli/pgcli,faif/python-patterns,chriskiehl/Gooey'

In [11]:
model.predict([df['repo'][42]])

[['fastai/fastai',
  'shengqiangzhang/examples-of-web-crawlers',
  'Miserlou/Zappa',
  'ageron/handson-ml2',
  'sherlock-project/sherlock',
  'donnemartin/interactive-coding-challenges',
  'weskerfoot/DeleteFB',
  'dae/anki',
  'scikit-learn/scikit-learn',
  'sloria/TextBlob',
  'deepfakes/faceswap',
  'smacke/subsync']]

### Under the Hood

In [12]:
cv = CountVectorizer(tokenizer=lambda x: x.split(','), max_features=1000)
X = cv.fit_transform(df['repo'])
X.todense()[:5]

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
nn = NearestNeighbors(n_neighbors=10)

In [14]:
nn.fit(X)
dist, ind = nn.kneighbors()
ind[:5]

array([[106,  14,  99,  48,  65,   3,  10,  19,   2,  40],
       [ 14,  66,  99,  10,  19,  65,  48,   3,  40,  12],
       [ 10,  19,   3,  99,  48,  14,  65,  40,  66,  92],
       [ 48,  10,  99,  19,  14,  65,   2,  40,  92,  72],
       [ 28,  10,  48,  72,  20,  19,  99,  14,  65,   3]])

#### Pickle 

In [15]:
import pickle

In [16]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

PicklingError: Can't pickle <function NNRecommender.<lambda> at 0x121f351e0>: attribute lookup NNRecommender.<lambda> on __main__ failed

In [17]:
import dill # pip install dill

with open('model.pkl', 'wb') as f:
    dill.dump(model, f)

del model

with open('model.pkl', 'rb') as f:
    model = dill.load(f)

In [18]:
model.predict(['streamlit/streamlit,huggingface/transformers,encode/httpx,aws/chalice,maxhumber/chart'])

[['shengqiangzhang/examples-of-web-crawlers',
  'mherrmann/fbs',
  'PySimpleGUI/PySimpleGUI',
  'scikit-learn/scikit-learn',
  'donnemartin/system-design-primer',
  'fastai/fastai',
  'sloria/TextBlob',
  'plasticityai/supersqlite',
  'minimaxir/big-list-of-naughty-strings',
  'TheAlgorithms/Python',
  'ytdl-org/youtube-dl',
  'vinta/awesome-python',
  'Avik-Jain/100-Days-Of-ML-Code']]